In [2]:
pip install chainladder

  Using cached chainladder-0.8.23-py3-none-any.whl.metadata (3.2 kB)
  Using cached sparse-0.15.4-py2.py3-none-any.whl.metadata (4.5 kB)
Using cached chainladder-0.8.23-py3-none-any.whl (1.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 21.3 MB/s eta 0:00:0000:0100:01
Using cached sparse-0.15.4-py2.py3-none-any.whl (237 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 3.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 34.2 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [47]:
import chainladder as cl
import numpy as np
import pandas as pd

In [39]:
comauto = cl.load_sample("clrd").groupby("LOB").sum().loc["comauto"]
print(comauto)
print(comauto["IncurLoss"])
print(comauto["EarnedPremNet"])

                                             Triangle Summary
Valuation:                                            1997-12
Grain:                                                   OYDY
Shape:                                         (1, 6, 10, 10)
Index:                                                  [LOB]
Columns:    [IncurLoss, CumPaidLoss, BulkLoss, EarnedPremD...
           12        24        36        48        60        72        84        96        108       120
1988  600839.0  632654.0  645663.0  646526.0  642177.0  639764.0  634631.0  631044.0  629857.0  629146.0
1989  676688.0  694520.0  703946.0  704996.0  695533.0  689237.0  684654.0  681943.0  679999.0       NaN
1990  741423.0  751063.0  748965.0  747312.0  743549.0  739173.0  732767.0  731394.0       NaN       NaN
1991  757620.0  768456.0  766137.0  755428.0  749200.0  739830.0  735818.0       NaN       NaN       NaN
1992  809703.0  811170.0  801484.0  784457.0  775180.0  772221.0       NaN       NaN       NaN       Na

In [57]:
comauto_1996 = comauto[comauto.valuation< pd.DatetimeIndex(["12/31/1997 23:59:59","12/31/1997 23:59:59"]).max()]
ia_1996 = cl.IncrementalAdditive(
    average = "simple",
    n_periods = [4,3,4,4,4,4,3,2,1]
    ).fit(
    X=comauto_1996["IncurLoss"], 
    sample_weight=comauto_1996["EarnedPremNet"].latest_diagonal)
ia_1996.incremental_

,12,24,36,48,60,72,84,96,108
1988,"600,839","31,815","13,009",863,"-4,349","-2,413","-5,133","-3,587","-1,187"
1989,"676,688","17,832","9,426","1,050","-9,463","-6,296","-4,583","-2,711","-1,253"
1990,"741,423","9,640","-2,098","-1,653","-3,763","-4,376","-6,406","-3,551","-1,369"
1991,"757,620","10,836","-2,319","-10,709","-6,228","-9,370","-6,157","-3,783","-1,459"
1992,"809,703","1,467","-9,686","-17,027","-9,277","-6,339","-6,416","-3,942","-1,520"
1993,"862,399","-16,150","5,131","-13,826","-8,149","-6,574","-6,655","-4,089","-1,577"
1994,"934,383","-21,172","-1,050","-12,131","-8,794","-7,095","-7,181","-4,412","-1,702"
1995,"958,551","-15,351","-2,310","-12,376","-8,972","-7,238","-7,327","-4,502","-1,736"
1996,"921,853","-18,545","-2,348","-12,580","-9,120","-7,357","-7,447","-4,576","-1,765"


In [58]:
ia_1997 = cl.IncrementalAdditive(
    average = "simple",
    n_periods = 4
    ).fit(
    X=comauto["IncurLoss"], 
    sample_weight=comauto["EarnedPremNet"].latest_diagonal)
ia.incremental_

,12,24,36,48,60,72,84,96,108,120
1988,"600,839","31,815","13,009",863,"-4,349","-2,413","-5,133","-3,587","-1,187",-711
1989,"676,688","17,832","9,426","1,050","-9,463","-6,296","-4,583","-2,711","-1,944",-750
1990,"741,423","9,640","-2,098","-1,653","-3,763","-4,376","-6,406","-1,373","-1,747",-820
1991,"757,620","10,836","-2,319","-10,709","-6,228","-9,370","-4,012","-3,010","-1,861",-874
1992,"809,703","1,467","-9,686","-17,027","-9,277","-2,959","-5,858","-3,136","-1,940",-911
1993,"862,399","-16,150","5,131","-13,826","-3,469","-6,540","-6,075","-3,253","-2,012",-944
1994,"934,383","-21,172","-1,050","3,055","-6,517","-7,058","-6,556","-3,510","-2,171","-1,019"
1995,"958,551","-15,351","-14,380","-11,073","-6,648","-7,200","-6,688","-3,581","-2,215","-1,040"
1996,"921,853","-16,960","-5,301","-11,255","-6,758","-7,319","-6,799","-3,640","-2,251","-1,057"
1997,"919,646","-18,303","-5,347","-11,351","-6,816","-7,381","-6,857","-3,671","-2,271","-1,066"
